## Artificial Neural Network Assignment

### Importing the modules

In [103]:
import pandas as pd
import numpy as np
np.set_printoptions(legacy="1.25")
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

### Importing and preproccessing data

In [104]:
data = pd.read_csv("updated_air_quality_data.csv")
data.drop("Unique ID", axis=1, inplace=True) # Removing the Unique ID column as it is not needed
data.dropna(inplace=True) # Remove any NaN if there
data

,Name,Measure,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value,Air Quality Category
0,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
1,Emissions,Density,UHF42,Unknown,Other,2015-01-01,1.2,Good
2,General Pollution,Miles,UHF42,Unknown,Annual Average,2011-12-01,8.6,Good
3,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
4,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...
18011,General Pollution,Miles,CD,Unknown,Other,2019-01-01,12.9,Moderate
18012,General Pollution,Miles,CD,Unknown,Other,2010-01-01,14.7,Moderate
18013,General Pollution,Miles,UHF42,Unknown,Other,2010-01-01,43.4,Bad
18014,General Pollution,Miles,UHF42,Queens,Other,2010-01-01,65.8,Bad


In [105]:
data_enc = data.copy()
for col in data:
	if (col != 'Start_Date') and (col != 'Data Value'):
		data_enc[col] = data[col].astype("category").cat.codes
data_enc["Start_Date"] = data["Start_Date"].apply(lambda a: int(a.replace("-", "")[2:])/10000) # coverting data by simply removing the first 2 digits of the year and then combining to yyddmm and then making the number small
print(dict(zip(data["Air Quality Category"].astype("category").cat.categories, data["Air Quality Category"].astype("category").cat.codes.unique()))) # print the mapping for the air quality
data_enc

{'Bad': 1, 'Good': 2, 'Moderate': 0}


,Name,Measure,Geo Type Name,Geo Place Name,Time Period,Start_Date,Data Value,Air Quality Category
0,1,1,4,3,1,15.0101,0.3,1
1,1,1,4,5,1,15.0101,1.2,1
2,2,2,4,5,0,11.1201,8.6,1
3,2,2,4,3,0,11.1201,8.0,1
4,2,2,4,3,2,22.0601,6.1,1
...,...,...,...,...,...,...,...,...
18011,2,2,1,5,1,19.0101,12.9,2
18012,2,2,1,5,1,10.0101,14.7,2
18013,2,2,4,5,1,10.0101,43.4,0
18014,2,2,4,3,1,10.0101,65.8,0


### Splitting the data

In [106]:
train_val_data = data_enc.sample(frac=0.85) # training + validation data
test_data = data_enc.drop(train_val_data.index)

train_val_target = train_val_data["Air Quality Category"] # seperate the target column
train_val_data.drop("Air Quality Category", axis=1, inplace=True)
test_target = test_data["Air Quality Category"] # seperate the target column
test_data.drop("Air Quality Category", axis=1, inplace=True)

train_val_target = pd.get_dummies(train_val_target).values # converting to one hot encoding
test_target = pd.get_dummies(test_target).values # same^

### Creating the structure of the Neural net then compiling it

In [182]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=len(train_val_data.columns)))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=3, activation='sigmoid')) # output layer. 3 units for each of good, bad, moderate (one-hot endcoded)

c:\Program Files\Python313\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [183]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

### Fitting the model

In [184]:
model.fit(train_val_data, train_val_target, validation_split=(15/85), batch_size=10, epochs=20)

Epoch 1/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - accuracy: 0.7365 - loss: 0.3642 - val_accuracy: 0.8542 - val_loss: 0.2249
Epoch 2/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 700us/step - accuracy: 0.8650 - loss: 0.2013 - val_accuracy: 0.8661 - val_loss: 0.1921
Epoch 3/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 692us/step - accuracy: 0.8821 - loss: 0.1774 - val_accuracy: 0.8994 - val_loss: 0.1578
Epoch 4/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 651us/step - accuracy: 0.8960 - loss: 0.1578 - val_accuracy: 0.9020 - val_loss: 0.1469
Epoch 5/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 629us/step - accuracy: 0.9079 - loss: 0.1425 - val_accuracy: 0.9219 - val_loss: 0.1293
Epoch 6/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 631us/step - accuracy: 0.9152 - loss: 0.1323 - val_accuracy: 0.7980 - val_loss: 0.4015
Epoch 7/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 636us/step - accuracy: 0.9205 - loss: 0.1249 - val_accuracy: 0.9415 - val_loss: 0.1055
Epoch 8/20
1262/1262 ━━━━━━━━━━━━━━━━━━━━ 1s 634us/step - accuracy: 0.9232 -

### Making predictions and calculating accuracy

In [185]:
pred_data = model.predict(test_data)
correct_pred_index = np.argmax(pred_data, axis=1)
correct_test_index = np.argmax(test_target, axis=1)
accuracy = np.mean(correct_pred_index == correct_test_index) * 100
accuracy

85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step


95.81791265729089

## Performance:

OF: Overfit
| Hidden Layers  | Activation  | Epochs  | Accuracy    |
| -------------- | ----------- | ------- | ----------- |
| 2 (8,8)        | sigmoid     | 20      | 90.52%      |
| 2 (8,8)        | relu + sig  | 10      | 88.71%      |
| 2 (8,8)        | relu + sig  | 20      | 94.97%      |
| 2 (8,8)        | relu + sig  | 25      | 92.26% (OF) |
| 2 (8,8)        | relu        | 20      | 29.34% (OF?)|
| 2 (16,16)      | relu + sig  | 11      | 92.78%      |
| 2 (16,16)      | relu + sig  | 20      | 88.78% (OF) |
| 2 (16,8)       | relu + sig  | 9       | 94.70%      |
| 3 (16,8,4)     | relu + sig  | 9       | 92.97%      |
| **3 (16,8,4)**     | **relu + sig**  | **20**      | **96.15%**      |
| 3 (16,8,4)     | relu + sig  | 25      | 93.56% (OF) |
| 3 (12,6,3)     | relu + sig  | 23      | 95.67%      |
| 3 (12,6,3)     | relu + sig  | 28      | 93.19% (OF) |